### RunnablePassthrough()

In [ ]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 모델 초기화
model = ChatOpenAI(model="gpt-4.1-nano")
prompt = ChatPromptTemplate.from_template(
    "{country}의 여행 할 만한 여행지를 3곳만 추천해줘."
)
chain1 = {"country": RunnablePassthrough()} | prompt | model | StrOutputParser()

print(chain1.invoke({"topic": "일본"}))

물론입니다! 일본에서 추천하는 여행지 3곳을 소개해드릴게요:

1. 교토 (Kyoto)  
전통과 현대가 어우러진 도시로, 기온 거리, 금각사(금閣寺), 기요미즈사(기요미즈데라) 등 아름다운 사찰과 전통 건축물을 즐기실 수 있습니다. 계절마다 다른 풍경과 일본 전통문화를 체험하기에 좋은 곳입니다.

2. 도쿄 (Tokyo)  
일본의 수도로서 현대적인 도시 풍경과 전통이 공존하는 곳입니다. 신주쿠, 시부야, 아사쿠사(센소지 사원) 등 다양한 명소와 쇼핑, 맛집, 엔터테인먼트 시설이 가득해 활기찬 분위기를 느끼실 수 있습니다.

3. 오사카 (Osaka)  
맛집과 활기찬 거리 분위기로 유명한 도시입니다. 오사카 성, 도톤보리 강가의 야간 풍경, 그리고 유니버설 스튜디오 일본(USJ) 등 다양한 엔터테인먼트 시설이 있어 즐거운 여행을 하실 수 있습니다.

이 세 곳은 일본의 다양한 매력을 느끼실 수 있는 대표적인 여행지입니다. 행복한 여행 되세요!


In [ ]:
prompt = ChatPromptTemplate.from_template(
    "김포공항에서 {country}을 직항으로 갈 수 있는 공항을 3개만 추천해줘"
)
chain2 = {"country": RunnablePassthrough()} | prompt | model | StrOutputParser()
print(chain2.invoke({"country": "일본"}))

김포공항에서 일본으로 직항 편이 운항되는 주요 공항 중 추천드릴 만한 공항은 다음과 같습니다:

1. 도쿄 하네다 공항 (HND)  
2. 오사카 간사이 공항 (KIX)  
3. 후쿠오카 공항 (FUK)  

이 공항들은 한국과 일본 간의 직항 노선이 많아 편리하게 이동하실 수 있습니다. 여행 일정과 항공사 소요 시간 등을 고려하여 선택하시면 좋겠습니다.


### RunnablePassthrough() + RunnableParallel()

In [ ]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
import json

# 모델 초기화 (gpt-4o-mini는 빠르고 저렴하여 테스트에 용이합니다)
model = ChatOpenAI(model="gpt-4o-mini")

# 1. 여행지 추천 체인
prompt1 = ChatPromptTemplate.from_template(
    "{country}의 여행할 만한 여행지를 3곳만 추천하고, 각 장소의 특징을 한 줄로 설명해줘."
)
chain1 = (
    # 입력이 문자열일 때, {"country": 입력값} 형태로 변환해 프롬프트에 전달
    {"country": RunnablePassthrough()}
    | prompt1
    | model
    | StrOutputParser()
)

# 2. 직항 공항 추천 체인
prompt2 = ChatPromptTemplate.from_template(
    "대한민국 김포공항에서 {country}(으)로 갈 수 있는 직항 공항을 3개만 알려줘."
)
chain2 = {"country": RunnablePassthrough()} | prompt2 | model | StrOutputParser()

# 3. RunnableParallel을 사용하여 두 체인 통합
# 'travel_info'와 'flight_info'라는 키로 각 체인의 결과를 받음
combined_chain = RunnableParallel(travel_info=chain1, flight_info=chain2)

combined_chain

{
  travel_info: {
                 country: RunnablePassthrough()
               }
               | ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 여행할 만한 여행지를 3곳만 추천하고, 각 장소의 특징을 한 줄로 설명해줘.'), additional_kwargs={})])
               | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000189AAF87090>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000189AAF73410>, root_client=<openai.OpenAI object at 0x00000189AAF86590>, root_async_client=<openai.AsyncOpenAI object at 0x00000189AAF86690>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
               | StrOutputParser(),
  flight_info: {
                 country: RunnablePassthrough()
               }
               | ChatPromptTemplate

In [ ]:
# 4. 통합된 체인 실행
# "일본"이라는 하나의 입력값이 chain1과 chain2에 동시에 전달됨
result = combined_chain.invoke("일본")

print(result["travel_info"])  # 여행지 추천 결과
print("=" * 70)
print(result["flight_info"])  # 직항 공항 추천 결과

1. **교토**: 전통적인 일본 문화와 아름다운 사찰, 정원이 가득한 도시로 역사적인 매력이 돋보임.

2. **후카이로**: 유유자적한 온천 마을로, 자연 속에서 뜨거운 온천욕을 즐기며 휴식을 취할 수 있음.

3. **홋카이도**: 사계절 내내 다양한 액티비티를 제공하는 지역으로, 특히 겨울철의 스키 리조트와 여름철의 아름다운 자연경관이 유명함.
김포공항에서 일본으로 갈 수 있는 직항 공항은 다음과 같습니다:

1. 도쿄 하네다 공항 (Tokyo Haneda Airport, HND)
2. 오사카 간사이 공항 (Osaka Kansai Airport, KIX)
3. 후쿠오카 공항 (Fukuoka Airport, FUK)

이 외에도 다른 일본 도시로 가는 직항이 있을 수 있지만, 위의 세 곳이 대표적인 직항 공항입니다.


### RunnableLambda

In [ ]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)
from langchain_core.output_parsers import StrOutputParser
import json
import random  # random 라이브러리 추가

# --- (이전 코드와 동일한 부분) ---

# 모델 초기화 (gpt-4o-mini는 빠르고 저렴하여 테스트에 용이합니다)
model = ChatOpenAI(model="gpt-4o-mini")

# 1. 여행지 추천 체인
prompt1 = ChatPromptTemplate.from_template(
    "{country}의 여행할 만한 여행지를 3곳만 추천하고, 각 장소의 특징을 한 줄로 설명해줘."
)
chain1 = {"country": RunnablePassthrough()} | prompt1 | model | StrOutputParser()

# 2. 직항 공항 추천 체인
prompt2 = ChatPromptTemplate.from_template(
    "대한민국 김포공항에서 {country}(으)로 갈 수 있는 직항 공항을 3개만 알려줘."
)
chain2 = {"country": RunnablePassthrough()} | prompt2 | model | StrOutputParser()

# 3. RunnableParallel을 사용하여 두 체인 통합
combined_chain = RunnableParallel(travel_info=chain1, flight_info=chain2)


# 4. 국가를 랜덤으로 선택하는 함수 정의
def choice_country(_):  # LangChain 체인과 연결하기 위해 입력 인자를 받도록 정의합니다.
    """주어진 리스트에서 국가를 하나 랜덤으로 선택합니다."""
    countries = ["미국", "영국", "프랑스", "일본", "스위스"]
    selected_country = random.choice(countries)

    # 어떤 나라가 선택되었는지 확인하기 위해 출력
    print(f"🌍 랜덤 선택된 나라: {selected_country}")

    return selected_country


# 5. RunnableLambda로 함수를 체인에 통합하고 최종 체인 완성
# [나라 선택 함수] -> [기존의 병렬 체인] 순서로 실행
final_chain = RunnableLambda(choice_country) | combined_chain

# 6. 최종 체인 실행
# 입력값이 필요 없으므로 None을 전달 (또는 비워둠)
result = final_chain.invoke(None)

# 7. 결과 출력
print("\n--- 최종 결과 ---")
print(json.dumps(result, indent=2, ensure_ascii=False))

🌍 랜덤 선택된 나라: 스위스

--- 최종 결과 ---
{
  "travel_info": "물론입니다! 스위스의 여행할 만한 장소 3곳을 추천합니다:\n\n1. **루체른(Lucerne)**: 아름다운 호수와 알프스 산맥의 경치를 배경으로 한 역사적인 도시로, 유명한 카펠교와 파노라마 경관이 매력입니다.\n\n2. **인터라켄(Interlaken)**: 호수와 산으로 둘러싸인 이곳은 다양한 액티비티와 함께 할 수 있는 베르너 오버란트 지역의 중심지로, 스키와 하이킹의 명소입니다.\n\n3. **제네바(Geneva)**: 국제기구들이 많은 글로벌 도시로, 아름다운 레만 호수와 다양한 문화적 명소가 어우러진 매력적인 장소입니다.",
  "flight_info": "김포공항에서 스위스로 갈 수 있는 직항 노선은 일반적으로 다음의 공항들로 연결됩니다:\n\n1. **취리히 공항 (Zurich Airport, ZRH)**: 스위스의 최대 도시인 취리히에 위치한 국제공항입니다.\n2. **제네바 공항 (Geneva Airport, GVA)**: 스위스의 두 번째로 큰 도시인 제네바에 위치한 국제공항입니다.\n3. **바젤-뮌헨 공항 (EuroAirport Basel-Mulhouse-Freiburg, BSL/MLH/EAP)**: 바젤, 뮌헨, 프라이부르크 지역에 서비스를 제공하는 공항입니다.\n\n다만, 항공사나 시즌에 따라 노선이 달라질 수 있으니, 정확한 정보를 위해서는 항공사나 공식 공항 웹사이트를 확인하는 것이 좋습니다."
}


In [ ]:
from operator import itemgetter

students_dict = [
    {"name": "홍길동", "age": 25, "score": 88},
    {"name": "이순신", "age": 32, "score": 95},
    {"name": "김유신", "age": 22, "score": 76},
]

# 딕셔너리의 'age' 키로 정렬
sorted_by_age = sorted(students_dict, key=itemgetter("age"))
print(f"\n딕셔너리 나이 정렬: {sorted_by_age}")

# 여러 항목(이름과 점수)을 한 번에 가져오기
get_name_and_score = itemgetter("name", "score")
for student in students_dict:
    print(f"가져온 항목: {get_name_and_score(student)}")


딕셔너리 나이 정렬: [{'name': '김유신', 'age': 22, 'score': 76}, {'name': '홍길동', 'age': 25, 'score': 88}, {'name': '이순신', 'age': 32, 'score': 95}]
가져온 항목: ('홍길동', 88)
가져온 항목: ('이순신', 95)
가져온 항목: ('김유신', 76)
